### **Library analogy (one-sentence)**

You have a set of tool-helpers (Wikipedia, arXiv, a local web crawler and a vector DB). An agent is a smart librarian that can call those tools, fetch sources, and then use a writer (LLM) to answer user questions by combining tool outputs.

In [32]:
!pip install arxiv

- **What it does:** Installs the Python package to query arXiv (research-paper search).
- **Analogy:** Buying a subscription so the librarian can look up the latest research.

#### **Wikipedia tool + utility**

In [33]:
!pip install wikipedia

In [34]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [35]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [36]:
wiki.name

'wikipedia'

- **What it does:** Creates a Wikipedia tool that will fetch up to 1 result and returns up to 200 characters per doc. `wiki` is the tool the agent can call.
- **Analogy:** A fast encyclopedia lookup the librarian can consult.

#### **Web page loader + splitting + embeddings + vector DB**

In [37]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings = OllamaEmbeddings(model="nomic-embed-text")  # fast embedding model
#vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020FE7300440>, search_kwargs={})

- **What it does:**
    - Loads the LangChain docs web page, splits the text into chunks (1000 characters each, 200 overlap), converts chunks into embeddings (fingerprints) using an Ollama embedding model, stores them in FAISS, and creates a `retriever` you can query for similar chunks.

- **Analogy:** The librarian photocopies pages, labels them with searchable fingerprints, and stores them in a fast cabinet for quick lookup.

- **Important note:** You correctly commented out the OpenAI embedding line. Keep using a dedicated embedding model like `nomic-embed-text`. Don’t use a chat LLM for embeddings — it’s slow and expensive.

#### **Convert retriever into a tool:**

In [38]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [39]:
retriever_tool.name

'langsmith_search'

- What it does: Wraps your retriever into a tool with a name and description so the agent can call it.
- Analogy: The librarian has a special drawer labeled “LangSmith docs” they can open and search.

#### **Arxiv Tool**

In [40]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

- What it does: Creates an arXiv query tool that fetches short results (1 result, 200 chars max).
- Analogy: A research alert service the librarian can query.

In [41]:
## Tools list
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\USER\\anaconda3\\envs\\textS\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000020FE4651E40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_comm

- **What it does:** Groups all tools the agent can use.
- **Analogy:** The librarian’s toolbox: encyclopedia, research database, internal docs cabinet.

#### **Load environment, LLM selection (OpenAI commented out later)**

In [42]:
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOllama(model="llama3.2:3b")

- **What it does:** Loads `.env` (if you plan to use OpenAI later). Creates a local Ollama LLM (`llama3.2:3b`) for generation. The OpenAI LLM line is present but commented out (you asked to keep it).
- Analogy: The librarian can either call an online expert (OpenAI) — commented out for now — or use a capable in-house expert (local Llama).

#### **Pull a prompt from LangChain hub**

In [48]:
from langchain import hub
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

# Pull the ReAct Prompt
# Get the ReAct prompt template from the hub
prompt = hub.pull("hwchase17/react")


- **What it does:** Downloads a prebuilt agent prompt (sets of instructions + function-call schema) from LangChain Hub.
- **Analogy:** You hand the librarian a prewritten template that says “when a user asks X, do Y.”

#### **Create an agent that can call external tools**

In [49]:
### Agents
from langchain.agents import create_openai_tools_agent
from langchain.agents import create_tool_calling_agent
from langchain.agents import create_react_agent

#agent=create_openai_tools_agent(llm,tools,prompt)
#agent = create_tool_calling_agent(llm, tools, prompt)
agent = create_react_agent(llm, tools, prompt)

- **What it does:** Builds an agent wired to the given LLM and tools, guided by the prompt. Note: the function is named `create_openai_tools_agent` but can often accept other LLMs (some versions expect an OpenAI LLM — if it errors, use a generic tools agent creator or the community-specific agent creation).

- `create_openai_tools_agent()` = built specifically for OpenAI-style models (`gpt-3.5-turbo`, `gpt-4-turbo`, etc.).
- `create_tool_calling_agent()` = general-purpose, works with Ollama, Anthropic, or any chat model that doesn’t strictly follow OpenAI’s schema.

- **Analogy:** You hire a librarian who knows when to consult which tool and how to synthesize answers.

#### **Wrap agent for execution**

In [50]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020FE7300440>, searc

- **What it does:** `AgentExecutor` manages the agent run — it handles tool calls, the LLM’s thinking, and returns answers. You then call `.invoke()` to ask questions.
- **Analogy:** Press a button and the librarian runs the whole process and returns the reply.

#### **Example queries**

In [51]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Question: Tell me about LangSmith
Thought: To answer this question, I should use the langsmith_search tool since it's specifically designed for searching information related to Langsmith.
Action: langsmith_search
Action Input: query='Langsmith'It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place.
LangSmith is framework agnostic, so you can use it with or without LangChain’s open-source libraries
langchain and langgraph.
Prototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI systems.
​Get started
Create an accountSign up at smith.langchain.com (no credit card required).
You can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.
Copy the key and save it securely.
Once your account and API key are ready, choose a quickstart to begin building with LangSmith:

Copy the key and save it securely.
On

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a framework-agnostic platform that provides tools for developing, debugging, and deploying Large Language Model (LLM) applications. It helps users trace requests, evaluate outputs, test prompts, and manage deployments in one place, offering observability, evaluation, prompt engineering, deployment, hosting, and workflow management capabilities.'}

In [52]:
agent_executor.invoke({"input":"What is Machine Learning?"})



> Entering new AgentExecutor chain...
Question: What is Machine Learning?

Thought: I need to find a general definition of Machine Learning that applies to various fields.

Action: wikipedia
Action Input: "Machine learning"
Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genQuestion: What is Machine Learning?
Thought: I need to find a general definition of Machine Learning that applies to various fields.

Action: wikipedia
Action Input: "Machine learning"
Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genI'll follow the format you specified.

Question: What is Machine Learning?
Thought: I need to find a general definition of Machine Learning that applies to various fields.

Action: wikipedi

{'input': 'What is Machine Learning?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [53]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
Action: arxiv
Action Input: "1605.08386"Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloQuestion: What's the paper 1605.08386 about?
Thought: I now know the final answer

Final Answer: The paper "Heat-bath random walks with Markov bases" (1605.08386) is about studying graphs on lattice points whose edges come from a finite set of allowed transitions, and it was published in 2016 by authors Caprice Stanley and Tobias Windisch.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper "Heat-bath random walks with Markov bases" (1605.08386) is about studying graphs on lattice points whose edges come from a finite set of allowed transitions, and it was published in 2016 by authors Caprice Stanley and Tobias Windisch.'}